In [1]:
import torch
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm import tqdm
%matplotlib inline

In [2]:
from cook_data import *

In [5]:
path ='/home/lupin-ya1/notebooks/DATA_INCOME/models/yolov5/data/KIC/00_pack.json'

In [6]:
labels = viaGetLables(path)[1]
labels

{'0_side': '0',
 '100_bills': '1',
 '200_bills': '2',
 '500_bills': '3',
 '1000_bills': '4',
 '2000_bills': '5',
 '5000_bills': '6',
 '0_box_closed': '7',
 '0_box': '8',
 '100_box_brown': '9',
 '200_box_green': '10',
 '500_box_violet': '11',
 '1000_box_turquoise': '12',
 '2000_box_blue': '13',
 '5000_box_orange': '14'}

In [7]:
out = via2yola(path, path[:-5], labels)['annotate']

convert via annotate for yola: 100%|██████████| 1000/1000 [00:00<00:00, 5642.19it/s]


In [9]:
path[:-5]

'/home/lupin-ya1/notebooks/DATA_INCOME/models/yolov5/data/KIC/00_pack'

In [10]:
for txt in out:
    with open(os.path.join(path[:-5], txt), 'w') as f:
        f.write('\n'.join(out[txt]))